# Method for Analysing and Quantifying the Intensity of ADF STEM Images

This notebook was first written March 2018

## Step 1 : Import hyperspy and atomap.

In [1]:
import matplotlib
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

In [2]:
import atomap.api as am

## Step 2: Import and analyse electron flux.

In [3]:
s = hs.load('PACBED.tif')
s.plot()
flux_pattern = s.data


The following step creates an interactive plot to select the region which to fit a power law to the flux patter. Until anything is selected 'profiler' will be empty intially.

In [4]:
profiler = am.quant.find_flux_limits(flux_pattern, 25)

Coordinates selected 59 670
Coordinates selected 59 410
Coordinates selected 214 410
Final coordinates are: 214, 410!
Coordinates selected 214 410


Select the left limit with the left mouse button, the right limit with the right mouse button and then confirm with the centre button.

Once coordinates have been seleted a message will be printed above confirming. Running analyse_flux fits a power law to the flux profile in the selected region.

In [7]:
expo = am.quant.analyse_flux((214,410), profiler[1], 25)
expo

(3.3006044767909, 472.22222222222223)

## Step 3: Import the an experimental image and a detector map.

In [8]:
s = hs.load('Pt0.7Ni0.3_ADF.tif')
s.plot()
det_image = am.example_data.get_detector_image_signal()

## Step 4: Normalised images to an absolute scale.

For detector quantification with flux weighting use:

In [12]:
s_normalised = am.quant.detector_normalisation (s,
                                                det_image,
                                                inner_angle=60,
                                                flux_expo=expo[0])
s_normalised.plot()

For detector quantification without flux weighting simply remove the flux exponent.

In [13]:
s_normalised2 = am.quant.detector_normalisation (s,
                                                det_image,
                                                inner_angle=60)
s_normalised2.plot()

## Step 5: Carry out peak finding on normalised image.

In [14]:
s_peaks = am.get_feature_separation(s, separation_range=(5,30), threshold_rel=0.1)
s_peaks.plot()

100%|██████████| 256/256 [00:00<00:00, 6027.65it/s]


In [15]:
atom_positions = am.get_atom_positions(s, 8, threshold_rel=0.1)

In [16]:
sublattice = am.Sublattice(atom_positions, s.data)
sublattice.get_atom_list_on_image().plot()

In [17]:
sublattice.construct_zone_axes()

In [18]:
sublattice.refine_atom_positions_using_2d_gaussian(sublattice.image)

Gaussian fitting: 100%|██████████| 256/256 [00:05<00:00, 46.42it/s]


## Step 6: Carry out integration of intensity over each atomic column.

In [26]:
(integrated_s, intensityRecord, pointRecord) = am.integrate(s_normalised, 
                                                            sublattice.x_position, 
                                                            sublattice.y_position, 
                                                            method= 'Voronoi',
                                                            max_radius=18)

In [27]:
h= hs.signals.Signal2D(pointRecord)
h.plot()

In [24]:
intensityRecord.plot(cmap='viridis')

Please note no multiplication is added during the integration step. No multiplication by pixel size or any normalisations.